[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/csikosmarci24/vitmav45-Mayuyu/tree/main/project/data_visualization.ipynb)

In [2]:
print('Hello Colab')

Hello Colab
